In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [ ]:
conn = pymongo.MongoClient()
cine = conn.cine
collection_cine2 = cine.collection_cine2

In [ ]:
collection_cine2.find_one()

### index
- 검색 속도를 향상시키기 위해서 사용하는 데이터 구조
- index 를 사용하지 않으면 collection 에 있는 document 를 
- 하나씩 조회하는 방식으로 검색함

### 기본 index
- MongDB 의 collection 은 기본적으로 _id key 기반으로 index 가 생성됨
  -  Single Field index (단일 필드 index)
  -  Compound Field index (복합 필드 index)

In [ ]:
'''
단일 필드 index : 하나의 field 로 index 를 생성함
 <-- 컬렉션이름.create_index(key(field))
'''
collection_cine2.create_index("배우 이름")

In [ ]:
'''
index 의 기본 정보 조회하기
  <-- 컬렉션이름.index_information()
'''
collection_cine2.index_information()

- key("필드이름", direction)
  -  direction
    -  pymongo.ASCENDING = 1 : 오름차순 (default)
    -  pymongo.DESCENDING = -1 : 내림차순
    -  pymongo.TEXT = "text" : text 검색을 쉽게 할 수 있도록 함

### index 삭제하기
- 전체 index 삭제
- 컬렉션이름.drop_indexes()
-  ㄴ 기본 index 이외의 모든 index 가 삭제됨

In [ ]:
collection_cine2.drop_indexes()

In [ ]:
collection_cine2.index_information()

- 특정 index 삭제하기
- 컬렉션이름.drop_index([('필드명',1)])

In [ ]:
collection_cine2.create_index("배우 이름")
collection_cine2.create_index("랭킹")
collection_cine2.create_index("흥행지수")

In [ ]:
collection_cine2.index_information()

In [ ]:
'''
"직업" key(field) 로 index 를 생성하면서
DESCENDING 으로 설정하기
'''
collection_cine2.create_index([("직업", pymongo.DESCENDING)])

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.create_index([("출연영화", "text")])

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.drop_indexes()

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.create_index("배우 이름")

In [ ]:
documents = \
  collection_cine2.find({"배우 이름":"김윤석"}, {"배우 이름":1, "출연영화":1, "_id":0})

for document in documents:
    print(document)

In [ ]:
import pprint

documents = \
  collection_cine2.find({"배우 이름":"김윤석"})

for document in documents:
    pprint.pprint(document)

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.create_index([("직업", "text")])

In [ ]:
collection_cine2.index_information()

In [ ]:
documents = collection_cine2.find({"$text":{"$search":"가수"}})

for document in documents:
    pprint.pprint(document)

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.drop_index([("배우 이름", 1)])

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.drop_index([("직업", "text")])

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.create_index([("배우 이름", "text")])


In [ ]:
collection_cine2.create_index([("출연영화", 1)])

In [ ]:
collection_cine2.drop_index([("출연영화", 1)])

In [ ]:
collection_cine2.index_information()

In [ ]:
collection_cine2.index_information()

In [ ]:
documents = collection_cine2.find({"$text" : {"$search":"박해일"}})
for document in documents:
    pprint.pprint(document)

In [ ]:
documents = collection_cine2.find({"$text" : {"$search":"김윤석"}})
for document in documents:
    pprint.pprint(document)

-  Compound Field index (복합 필드 index)
  - 최대 31 개의 필드로 index 를 생성할 수 있음
  - 여러 개의 필드를 하나의 index 로 지정함

In [ ]:
collection_cine2.drop_indexes()

In [ ]:
'''
복합 필드 인덱스 생성하기
'''
collection_cine2.create_index(
                                 [("출연영화",pymongo.TEXT), ("직업", pymongo.TEXT),("흥행지수", pymongo.DESCENDING)]                                 
                             )

In [ ]:
collection_cine2.index_information()

### 복합 필드 인덱스로 부분문자열 검색(조회)하기

In [ ]:
for document in collection_cine2.find({"$text":{"$search":"가수"}}):
    pprint.pprint(document)

In [ ]:
for document in collection_cine2.find({"$text":{"$search":"모가디슈"}}):
    pprint.pprint(document)

### 정규표현식(Regular Expression) 을 사용해서 부분문자열 검색하기

In [ ]:
'''
출연영화에 "함께"라는 문자열이 있는 document 조회하기
'''
for document in collection_cine2.find({"출연영화":{"$regex":"함께"}}):
    pprint.pprint(document)

In [ ]:
'''
출연영화에 "국가"라는 문자열이 있는 document 조회하기
'''
for document in collection_cine2.find({"출연영화":{"$regex":"국가"}}):
    pprint.pprint(document)

In [ ]:
'''
collection_text 라는 컬렉션 생성하기
'''
collection_text = cine.collection_text

In [ ]:
doc1 = {"name":"java cafe", "description":"Coffe & Cake", "ranking":1}
doc2 = {"name":"Burger Pot", "description":"Coffe & Burgar", "ranking":2}
doc3 = {"name":"Coffee Shop", "description":"Only Coffee", "ranking":3}
doc4 = {"name":"Clothes Shop", "description":"Discount Clothes", "ranking":4}
doc5 = {"name":"Shopping Site", "description":"Shopping & Rest", "ranking":5}

In [ ]:
collection_text.insert_many(
    [doc1, doc2, doc3, doc4, doc5]
)

In [ ]:
'''
regex
  .  문자 하나
  +  하나 이상
  *  없거나 또는 그 이상
'''

for document in collection_text.find({"name" : {"$regex" : "Co.+"}}, {"_id":0}):
    print(document)

In [ ]:
for document in collection_text.find({"name": {"$regex":"Co.*"}}, {"_id":0}):
    print(document)

In [ ]:
collection_text.insert_one(
    {"name":"Co", "description":"cafe", "ranking":6}
)

In [ ]:
for document in collection_text.find({"name": {"$regex":"Co.*"}}, {"_id":0}):
    print(document)

In [ ]:
for document in collection_text.find({"name": {"$regex":"Co.+"}}, {"_id":0}):
    print(document)

In [ ]:
'''
name 필드에 text 속성을 지정해서 index 생성하기
'''
collection_text.create_index([("name", "text")])

In [ ]:
collection_text.index_information()

In [ ]:
for document in collection_text.find({"$text":{"$search":"Coffee"}}, {"_id":0}):
    pprint.pprint(document)

In [ ]:
documents = \
  collection_text.find({"$text":{"$search":"Java coffee shop"}}, {"_id":0})

for document in documents:
    pprint.pprint(document)

In [ ]:
''' 
 "검색할문자열" : "" 속에 지정한 문자열과 일치하는 내용만 조회함
'''
documents = \
  collection_text.find({"$text":{"$search": "\"Java coffee shop\""}}, {"_id":0})

for document in documents:
    print(document)

In [ ]:
documents = \
  collection_text.find({"$text":{"$search": "\"coffee shop\""}}, {"_id":0})

for document in documents:
    print(document)

In [ ]:
documents = \
  collection_text.find({"$text":{"$search": "\"burger pot\""}}, {"_id":0})

for document in documents:
    print(document)

In [ ]:
'''
대소문자 구분해서 일치하는 내용 조회하기

  "$caseSensitive":True 
  
  라고 설정함
'''

documents = \
  collection_text.find({"$text":{"$search":"\"coffee shop\"", "$caseSensitive":True}}, {"_id":0})

for document in documents:
    print(document)

In [ ]:
for document in collection_cine2.find({}, {"학교":1, "배우 이름":1, "_id":0}):
    pprint.pprint(document)

In [ ]:
'''
서울예술대학 출신을 흥행지수 내림차순으로 10 명 조회하기
(정규표현식 사용하기)
출력필드 : 배우 이름, 학교, 흥행지수
'''
documents = collection_cine2.find({"학교":{"$regex":"서울예술대학"}},\
                                  {"배우 이름":1, "학교":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)

In [ ]:
documents = collection_cine2.find({"학교":{"$regex":"중앙대학교"}},\
                                  {"배우 이름":1, "학교":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)